# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [77]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [78]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [64]:
import numpy as np
from collections import Counter
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    vocab_to_int = { word:idx for idx ,word in enumerate(vocab)}
    int_to_vocab = { idx:word for idx ,word in enumerate(vocab)}
    
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [60]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation = ['.',',','"',';','!','?','(',')','--','\n']
    token = ['||Period||','||Comma||','||Quotation_Mark||','||Semicolon||','||Exclamation_mark||','||Question_mark||','Left_Parentheses','||Right_Parentheses||','||Dash||','||Return||']
    puct_token_dict = {punct:word for punct,word in zip(punctuation,token)}
   
    return puct_token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [59]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [79]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [57]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [56]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function

    input_ = tf.placeholder(tf.int32, [None,None], name='input')
    targets_ = tf.placeholder(tf.int32, [None,None], name='targets')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
      
    return (input_, targets_, learning_rate_)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [55]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    num_layers = 2
 
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm]* num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [37]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    embed = tf.nn.embedding_lookup(embedding,input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [38]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function

    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs,dtype=tf.float32)
    final_state = tf.identity(final_state,name='final_state')
    
    
    return (outputs,final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [65]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed_layer = get_embed(input_data, vocab_size, embed_dim)
    outputs,final_state  = build_rnn(cell, embed_layer)
    #print(final_state.shape)
    tf.identity(final_state, name='final_state')
    #print(outputs.shape)
    
    #logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    logits = tf.layers.dense(outputs, vocab_size, activation=None, use_bias=True)
   
    return (logits,final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [66]:

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    #np.array(int_text).reshape=(int_text,batch_size,seq_length)
    #print(type(int_text))
    characters_per_batch = batch_size*seq_length
    n_batches = len(int_text)//characters_per_batch
    
    # Keep only enough characters to make full batches

    x = np.array(int_text[ : (n_batches * characters_per_batch)])
    y = np.array(int_text[1 : (n_batches * characters_per_batch)+1])
    y[-1] = x[0]
    
    inputs = x.reshape(batch_size, -1)
    targets = y.reshape(batch_size, -1)

    inputs = np.split(inputs, n_batches,1)
    targets = np.split(targets, n_batches,1)
    
    batches = np.array(list(zip(inputs, targets)))
    return batches
        
    
    #return None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [76]:
# Number of Epochs
num_epochs = 250
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 128
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 32

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [67]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [54]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/134   train_loss = 8.822
Epoch   0 Batch   32/134   train_loss = 6.374
Epoch   0 Batch   64/134   train_loss = 6.442
Epoch   0 Batch   96/134   train_loss = 6.387
Epoch   0 Batch  128/134   train_loss = 6.241
Epoch   1 Batch   26/134   train_loss = 5.958
Epoch   1 Batch   58/134   train_loss = 6.387
Epoch   1 Batch   90/134   train_loss = 6.277
Epoch   1 Batch  122/134   train_loss = 6.191
Epoch   2 Batch   20/134   train_loss = 5.961
Epoch   2 Batch   52/134   train_loss = 5.788
Epoch   2 Batch   84/134   train_loss = 5.780
Epoch   2 Batch  116/134   train_loss = 5.769
Epoch   3 Batch   14/134   train_loss = 5.900
Epoch   3 Batch   46/134   train_loss = 5.707
Epoch   3 Batch   78/134   train_loss = 5.460
Epoch   3 Batch  110/134   train_loss = 5.827
Epoch   4 Batch    8/134   train_loss = 5.334
Epoch   4 Batch   40/134   train_loss = 5.500
Epoch   4 Batch   72/134   train_loss = 5.554
Epoch   4 Batch  104/134   train_loss = 5.446
Epoch   5 Batch    2/134   train_l

Epoch  42 Batch  100/134   train_loss = 3.489
Epoch  42 Batch  132/134   train_loss = 3.344
Epoch  43 Batch   30/134   train_loss = 3.451
Epoch  43 Batch   62/134   train_loss = 3.372
Epoch  43 Batch   94/134   train_loss = 3.500
Epoch  43 Batch  126/134   train_loss = 3.448
Epoch  44 Batch   24/134   train_loss = 3.483
Epoch  44 Batch   56/134   train_loss = 3.435
Epoch  44 Batch   88/134   train_loss = 3.428
Epoch  44 Batch  120/134   train_loss = 3.396
Epoch  45 Batch   18/134   train_loss = 3.278
Epoch  45 Batch   50/134   train_loss = 3.215
Epoch  45 Batch   82/134   train_loss = 3.204
Epoch  45 Batch  114/134   train_loss = 3.364
Epoch  46 Batch   12/134   train_loss = 3.355
Epoch  46 Batch   44/134   train_loss = 3.435
Epoch  46 Batch   76/134   train_loss = 3.161
Epoch  46 Batch  108/134   train_loss = 3.229
Epoch  47 Batch    6/134   train_loss = 3.217
Epoch  47 Batch   38/134   train_loss = 3.297
Epoch  47 Batch   70/134   train_loss = 3.137
Epoch  47 Batch  102/134   train_l

Epoch  85 Batch   66/134   train_loss = 2.151
Epoch  85 Batch   98/134   train_loss = 2.170
Epoch  85 Batch  130/134   train_loss = 2.132
Epoch  86 Batch   28/134   train_loss = 2.207
Epoch  86 Batch   60/134   train_loss = 2.258
Epoch  86 Batch   92/134   train_loss = 2.267
Epoch  86 Batch  124/134   train_loss = 2.095
Epoch  87 Batch   22/134   train_loss = 2.281
Epoch  87 Batch   54/134   train_loss = 2.272
Epoch  87 Batch   86/134   train_loss = 2.008
Epoch  87 Batch  118/134   train_loss = 2.249
Epoch  88 Batch   16/134   train_loss = 2.167
Epoch  88 Batch   48/134   train_loss = 2.215
Epoch  88 Batch   80/134   train_loss = 1.920
Epoch  88 Batch  112/134   train_loss = 2.198
Epoch  89 Batch   10/134   train_loss = 2.339
Epoch  89 Batch   42/134   train_loss = 2.146
Epoch  89 Batch   74/134   train_loss = 2.063
Epoch  89 Batch  106/134   train_loss = 2.290
Epoch  90 Batch    4/134   train_loss = 2.095
Epoch  90 Batch   36/134   train_loss = 2.165
Epoch  90 Batch   68/134   train_l

Epoch 128 Batch   32/134   train_loss = 1.726
Epoch 128 Batch   64/134   train_loss = 1.657
Epoch 128 Batch   96/134   train_loss = 1.738
Epoch 128 Batch  128/134   train_loss = 1.497
Epoch 129 Batch   26/134   train_loss = 1.613
Epoch 129 Batch   58/134   train_loss = 1.563
Epoch 129 Batch   90/134   train_loss = 1.495
Epoch 129 Batch  122/134   train_loss = 1.454
Epoch 130 Batch   20/134   train_loss = 1.713
Epoch 130 Batch   52/134   train_loss = 1.732
Epoch 130 Batch   84/134   train_loss = 1.778
Epoch 130 Batch  116/134   train_loss = 1.522
Epoch 131 Batch   14/134   train_loss = 1.579
Epoch 131 Batch   46/134   train_loss = 1.390
Epoch 131 Batch   78/134   train_loss = 1.482
Epoch 131 Batch  110/134   train_loss = 1.585
Epoch 132 Batch    8/134   train_loss = 1.608
Epoch 132 Batch   40/134   train_loss = 1.548
Epoch 132 Batch   72/134   train_loss = 1.400
Epoch 132 Batch  104/134   train_loss = 1.430
Epoch 133 Batch    2/134   train_loss = 1.560
Epoch 133 Batch   34/134   train_l

Epoch 170 Batch  132/134   train_loss = 1.135
Epoch 171 Batch   30/134   train_loss = 1.123
Epoch 171 Batch   62/134   train_loss = 1.048
Epoch 171 Batch   94/134   train_loss = 1.124
Epoch 171 Batch  126/134   train_loss = 1.243
Epoch 172 Batch   24/134   train_loss = 1.316
Epoch 172 Batch   56/134   train_loss = 1.114
Epoch 172 Batch   88/134   train_loss = 1.087
Epoch 172 Batch  120/134   train_loss = 1.092
Epoch 173 Batch   18/134   train_loss = 1.027
Epoch 173 Batch   50/134   train_loss = 1.100
Epoch 173 Batch   82/134   train_loss = 0.991
Epoch 173 Batch  114/134   train_loss = 1.155
Epoch 174 Batch   12/134   train_loss = 1.105
Epoch 174 Batch   44/134   train_loss = 1.015
Epoch 174 Batch   76/134   train_loss = 1.086
Epoch 174 Batch  108/134   train_loss = 1.112
Epoch 175 Batch    6/134   train_loss = 1.014
Epoch 175 Batch   38/134   train_loss = 1.031
Epoch 175 Batch   70/134   train_loss = 1.246
Epoch 175 Batch  102/134   train_loss = 1.237
Epoch 176 Batch    0/134   train_l

Epoch 213 Batch   98/134   train_loss = 0.851
Epoch 213 Batch  130/134   train_loss = 0.761
Epoch 214 Batch   28/134   train_loss = 0.818
Epoch 214 Batch   60/134   train_loss = 0.938
Epoch 214 Batch   92/134   train_loss = 0.844
Epoch 214 Batch  124/134   train_loss = 0.732
Epoch 215 Batch   22/134   train_loss = 0.911
Epoch 215 Batch   54/134   train_loss = 0.845
Epoch 215 Batch   86/134   train_loss = 0.704
Epoch 215 Batch  118/134   train_loss = 0.923
Epoch 216 Batch   16/134   train_loss = 0.755
Epoch 216 Batch   48/134   train_loss = 0.907
Epoch 216 Batch   80/134   train_loss = 0.857
Epoch 216 Batch  112/134   train_loss = 0.900
Epoch 217 Batch   10/134   train_loss = 0.885
Epoch 217 Batch   42/134   train_loss = 0.778
Epoch 217 Batch   74/134   train_loss = 0.819
Epoch 217 Batch  106/134   train_loss = 0.930
Epoch 218 Batch    4/134   train_loss = 0.742
Epoch 218 Batch   36/134   train_loss = 0.717
Epoch 218 Batch   68/134   train_loss = 0.785
Epoch 218 Batch  100/134   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [70]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_ = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
        
    
    
    
    
    return input_, initial_state, final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [74]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    
    """
    #print(list(int_to_vocab.values()))
    #print(probabilities)
    # TODO: Implement Function
    c = np.random.choice(list(int_to_vocab.values()),1, p=probabilities)[0]
    return c


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [75]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: wait, let me look here for to happen with your feet of because she asked ya!
moe_szyslak: well, god, two big turkey, just sorry mr. burns, rummy!
homer_simpson: no, i don't know.
homer_simpson: thanks, moe...
moe_szyslak: huh... god, you... six...(gives. maybe he watch, get a-- the insurance break moe comedy then eating town like it was our" face"" rummy love like me?" goodbye, and" is it, gimme the pants.
moe_szyslak: no.
homer_simpson: you know, i used to say to less hair i want to turn back in the world.
moe_szyslak: the two of my place quiet.
wally: i don't get drunk. it's bright in the two of a hobo!
carl_carlson: why don't we don't let him back.
moe_szyslak: you know, you're tired of god, i can think ya just work?
homer_simpson: a cooler, isn't a lot.
well my


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.